In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18253494
paper_name = 'cook_ho_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [10]:
original_data = pd.read_excel('raw_data/pone.0001546.s015.xlsx', sheet_name='Sheet1', header=[0,1])

In [11]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4651 x 18


In [12]:
original_data.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Exp 1 - 24 ml/min           \
                 ORF               Name                UP       DN   
0            YIL134W               FLX1          -5.04366      NaN   
1            YNR068C                NaN          -5.78244 -5.83223   
2            YHR177W                NaN           -4.8244 -5.18674   
3            YER111C               SWI4          -6.79369  -2.5688   
4            YIL154C              IMP2'               NaN      NaN   

  Exp 2 - 24 ml/min          Exp 2 (average) - 21 ml/min           \
                 UP       DN                          UP       DN   
0          -6.14723      NaN                    -4.46741      NaN   
1          -6.76548  -6.0021                     -4.8086 -4.74011   
2          -4.25115 -8.89745                    -4.04819 -3.97647   
3               -10 -4.04553                    -6.00561 -5.00752   
4               NaN      -10                   -0.753583 -4.98514   

  Exp 3 - 16 ml/min (average)          Exp 3 - 24 ml/min (average)           \
                           UP       DN                          UP       DN   
0                    -5.28773      NaN                    -6.23994      NaN   
1                    -4.52202 -4.64037                    -5.32549 -5.03435   
2                    -5.24499 -5.65722                    -5.39264 -5.86538   
3                    -3.73092 -3.45563                    -5.32274 -3.96127   
4                         NaN -4.63905                         NaN -5.06264   

                                            \
  Average across all arrays and UP/DN tags   
0                                -5.437195   
1                                -5.345320   
2                                -5.334465   
3                                -5.089171   
4                                -5.088084   

                                              \
  Average across all arrays and UP/DN tags.1   
0                                        LGE   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   

                                              \
  Average across all arrays and UP/DN tags.2   
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   

                                              \
  Average across all arrays and UP/DN tags.3   
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   

                                                      \
          Average across all arrays and UP/DN tags.4   
0                                                NaN   
1                                                NaN   
2                      X = no tag in this collection   
3                    blank = signal below thresholds   
4  Value of 10 or -10 [arbitrarily set; significa...   

                                              
  Average across all arrays and UP/DN tags.5  
0                                        NaN  
1                                        NaN  
2                                        NaN  
3                                        NaN  
4                                        NaN

In [13]:
original_data['orf'] = original_data[('Unnamed: 0_level_0','ORF')].astype(str)

In [14]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [15]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [16]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [(Unnamed: 0_level_0, ORF), (Unnamed: 1_level_0, Name), (Exp 1 - 24 ml/min, UP), (Exp 1 - 24 ml/min, DN), (Exp 2 - 24 ml/min, UP), (Exp 2 - 24 ml/min, DN), (Exp 2 (average) - 21 ml/min, UP), (Exp 2 (average) - 21 ml/min, DN), (Exp 3 - 16 ml/min (average), UP), (Exp 3 - 16 ml/min (average), DN), (Exp 3 - 24 ml/min (average), UP), (Exp 3 - 24 ml/min (average), DN), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags.1), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags.2), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags.3), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags.4), (Exp 3 - 24 ml/min (average), Average across all arrays and UP/DN tags.5), (orf, )]
Index: []


In [20]:
# Seems that, originally, negative values = large cells, positive values = small cells. By convention, we revert that.
original_data['data'] = -original_data[('Exp 3 - 24 ml/min (average)', 'Average across all arrays and UP/DN tags')].astype(float)

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(4627, 1)

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [5374]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,5374
data_type,value
orf,
YAL002W,1.344456
YAL004W,0.382591
YAL005C,-0.009773
YAL007C,0.206225
YAL009W,-2.217163


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5374
,data_type,value
gene_id,orf,
2,YAL002W,1.344456
1863,YAL004W,0.382591
4,YAL005C,-0.009773
5,YAL007C,0.206225
7,YAL009W,-2.217163


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id           5374          
data_type           value    valuez
gene_id orf                        
2       YAL002W  1.344456  1.602666
1863    YAL004W  0.382591  0.507620
4       YAL005C -0.009773  0.060928
5       YAL007C  0.206225  0.306834
7       YAL009W -2.217163 -2.452099

# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18253494...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.99s/it]

Updating the data_modified_on field...
